# Part 9. 데이터 결합 (merge / join / concat)
이 장은 하나의 테이블로는 부족해지는 순간을 다룬다. <br>
분석이 깊어질수록 데이터는 반드시 "붙이는 단계"를 거친다.

## 1) 데이터 결합이 필요한 이유
현실의 데이터는 하나의 테이블에 전부 담기지 않는다.<br>
보통 데이터는 다음처럼 나뉘어 있다.
- 거래/로그 테이블 → 언제, 무엇을 했는가
- 기준/마스터 테이블 → 그 값이 무엇을 의미하는가

분석을 하려면 이 정보들을 하나의 기준으로 연결해야 한다.

## 2) merge의 핵심 개념
```python
왼쪽_데이터프레임.merge(
    오른쪽_데이터프레임,
    on="공통_키",
    how="결합_방식")

```

merge는 공통 키(key)를 기준으로 테이블을 옆으로 붙인다.<br>
그래서 키 선택이 틀리면 에러 없이 실행되어도 결과 전체가 틀어질 수 있다.


핵심 포인트
- 행을 늘리는 게 아니다.
- 컬럼을 늘리는 작업이다.

In [2]:
import pandas as pd

orders = pd.DataFrame([
    {"order_id": 1, "date": "2026-01-01", "menu": "Latte",     "sales": 5000, "customer_id": "C01"},
    {"order_id": 2, "date": "2026-01-01", "menu": "Americano", "sales": 4500, "customer_id": "C02"},
    {"order_id": 3, "date": "2026-01-02", "menu": "Cake",      "sales": 6000, "customer_id": "C03"},
])

menu_map = pd.DataFrame([
    {"menu": "Latte",     "category": "Coffee"},
    {"menu": "Americano", "category": "Coffee"},
    # Cake는 매핑 누락 상태
])

customers = pd.DataFrame([
    {"customer_id": "C01", "city": "Suwon",  "grade": "VIP"},
    {"customer_id": "C02", "city": "Yongin", "grade": "NEW"},
    {"customer_id": "C03", "city": "Suwon",  "grade": "NORMAL"},
])
print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 고객 정보 데이터")
print(customers)

주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 고객 정보 데이터
  customer_id    city   grade
0         C01   Suwon     VIP
1         C02  Yongin     NEW
2         C03   Suwon  NORMAL


In [3]:
merged = orders.merge(
    menu_map,
    on="menu",
    how="left"
)

print(merged)

   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


orders를 기준으로
- menu를 키로
- menu_map의 category를 붙인다

merge의 결과로
- orders에는 없던 category 컬럼이 생긴다
- Cake처럼 매핑이 없는 값은 결측값이 된다

## 3) how 옵션의 의미
how는 결합 후에 어떤 행(row)을 남길지 결정하는 옵션이다. <br>
how = “기준을 어디에 두고 행을 남길 것인가”

- left<br>
→ 왼쪽 테이블의 행은 전부 유지

- inner<br>
→ 양쪽 테이블에 모두 존재하는 키만 유지

- outer<br>
→ 양쪽 테이블의 모든 행을 유지

### left — 왼쪽(원본) 유지 [LEFT JOIN]

```python
왼쪽.merge(오른쪽, on=키, how="left")

```
- 왼쪽 데이터프레임의 행은 전부 유지
- 오른쪽에 매핑이 없으면 NaN

언제 사용하나?
- 실무에서 가장 많이 쓰는 옵션
- 원본 로그는 보존하고
- 매핑 누락을 NaN으로 확인할 때

In [4]:
left_join = orders.merge(menu_map, on="menu", how="left")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (LEFT)")
print(left_join)

주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (LEFT)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


### inner — 양쪽에 모두 있는 것만 [INNER JOIN]
```python
왼쪽.merge(오른쪽, on=키, how="inner")

```
- 양쪽 테이블에 공통으로 존재하는 키만 남김
- 매핑 안 되는 행은 아예 제거됨

언제 사용하나?
- 매핑이 제대로 됐는지
- 정합성 확인할 때

In [5]:
inner_join = orders.merge(menu_map, on="menu", how="inner")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (INNER)")
print(inner_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (INNER)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee


### outer — 전체 합치기
```python
왼쪽.merge(오른쪽, on=키, how="outer")

```
- 양쪽 테이블의 모든 행을 전부 유지
- 한쪽에만 있으면 NaN

언제 사용하나?
- 어느 쪽에만 있는 값이 있는지
- 누락 / 신규 항목 탐색할 때

In [6]:
outer_join = orders.merge(menu_map, on="menu", how="outer")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (outer)")
print(outer_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (outer)
   order_id        date       menu  sales customer_id category
0         2  2026-01-01  Americano   4500         C02   Coffee
1         3  2026-01-02       Cake   6000         C03      NaN
2         1  2026-01-01      Latte   5000         C01   Coffee


## 4) merge vs join vs concat
이 세 개는 목적이 다르다.

1. merge → 키 기준 옆으로 붙이기
2. join → 인덱스 기준 결합 (merge의 축약형)
3. concat → 위아래로 이어 붙이기

전처리·분석 단계에서는 merge가 표준이다.

## 5) 결합 후 반드시 해야하는것
데이터 결합이 끝났다고 바로 분석하면 위험하다.<br>

반드시 확인해야 할 것
- 매핑되지 않은 행이 있는가?
- 예상보다 행 수가 늘거나 줄지 않았는가?
- NaN이 갑자기 늘지 않았는가?

이를 확인하기 위한 가장 좋은 도구가 indicator 옵션이다.

indicator를 사용하면 각 행이 어떤 방식으로 결합되었는지 한눈에 알 수 있다.
- 왼쪽에만 있던 행
- 오른쪽에만 있던 행
- 정상적으로 매칭된 행

결합 결과에서 "어디서 문제가 생겼는지"를 바로 확인할 수 있다.

In [7]:
check = orders.merge(
    menu_map,
    on="menu",
    how="left",
    indicator=True
)

missing = check[check["_merge"] == "left_only"]
print(missing[["menu", "order_id"]])

   menu  order_id
2  Cake         3


In [8]:
check

,order_id,date,menu,sales,customer_id,category,_merge
0,1,2026-01-01,Latte,5000,C01,Coffee,both
1,2,2026-01-01,Americano,4500,C02,Coffee,both
2,3,2026-01-02,Cake,6000,C03,NaN,left_only


indicator의 의미
- both → 양쪽에 있음 → 정상적으로 매핑된 데이터
- left_only → 왼쪽에만 있음 (매핑 누락)
- right_only → 오른쪽에만 있음 → 현재 분석 대상에는 없는 데이터